In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

In [36]:
# Data preparation
%run manageDataAmaury.ipynb
trainingSetValues = trainingSetValues
trainingSetLabels = trainingSetLabels
testSetValues = testSetValues

In [ ]:
# analysis of data quality of the data set
# pandas_profiling.ProfileReport(trainingSetValues)

Pas besoin de supprimer des variables

In [ ]:
# correlation maxtrix
cor = trainingSetValues.corr()
sns.heatmap(cor, square=True, cmap="coolwarm", annot=True)

In [ ]:
# transformation of id in index
pd.DataFrame.set_index(trainingSetLabels, 'id', inplace=True)
# trainingSetValues.drop(columns='population', inplace=True)
# testSetValues.drop(columns='population', inplace=True)

In [ ]:
# encoding of categorical features
from sklearn.preprocessing import LabelEncoder

def encodeCategoricalFeature(dfTrain, dfTest):
    features = trainingSetValues.columns
    numericalFeatures = trainingSetValues._get_numeric_data().columns
    categoricalFeatures = list(set(features) - set(numericalFeatures))
    enc = LabelEncoder()
    for feature in categoricalFeatures:
        enc.fit(pd.concat([dfTest[feature], dfTrain[feature]]))
        dfTrain[feature] = enc.transform(dfTrain[feature])
        dfTest[feature] = enc.transform(dfTest[feature])

encodeCategoricalFeature(trainingSetValues, testSetValues)
#pandas_profiling.ProfileReport(trainingSetValues)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(trainingSetValues, trainingSetLabels, test_size=0.25, random_state=0)

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer
parameters = {'n_estimators':[100,500,1000], 'max_features':('sqrt', 'log2', None)}
rf = RandomForestClassifier(parameters, random_state=66)
y = np.ravel(y_train)
x = x_train
models = GridSearchCV(rf, param_grid=parameters, scoring=make_scorer(accuracy_score),cv=3)
models.fit(x, y)

GridSearchCV(cv=3,
             estimator=RandomForestClassifier(n_estimators={'max_features': ('sqrt',
                                                                             'log2',
                                                                             None),
                                                            'n_estimators': [100,
                                                                             500,
                                                                             1000]},
                                              random_state=66),
             param_grid={'max_features': ('sqrt', 'log2', None),
                         'n_estimators': [100, 500, 1000]},
             scoring=make_scorer(accuracy_score))

In [25]:
argModel = np.argmin(models.cv_results_['rank_test_score'])
paramModel = models.cv_results_['params'][argModel]
print('Parameters of our best model: ' + str(paramModel))
model = models.best_estimator_

Parameters of our best model: {'max_features': 'sqrt', 'n_estimators': 1000}


In [26]:
predictions = model.predict(x_test)
score = accuracy_score(predictions, y_test)
print('Accuracy score on x_test: ' + str(score))

Accuracy score on x_test: 0.8049158249158249


In [27]:
# make prediction on new values
testPrediction = model.predict(testSetValues)
predictionDf = pd.DataFrame()
predictionDf['id'] = testSetValues.index
predictionDf['status_group'] = testPrediction
pd.DataFrame.set_index(predictionDf, 'id', inplace=True)
print(predictionDf)
predictionDf.to_csv('data/results.csv')

         status_group
id                   
50785  non functional
51630      functional
17168      functional
45559  non functional
49871      functional
...               ...
39307  non functional
18990      functional
28749      functional
33492      functional
68707  non functional

[14850 rows x 1 columns]
